Код составлен на основе туторила с Kaggle: [ссылка](https://www.kaggle.com/code/neerajmohan/fine-tuning-bert-for-text-classification)

In [1]:
!pip install transformers

# Обучение BERT для классификации

Задача: создать модель, которая по тексту сообщения будет определять, говорится в нем о катастрофе (например пожаре) или нет.

## Подгрузка данных

In [2]:
import numpy as np
import pandas as pd
import time
import datetime
import gc
import random
from nltk.corpus import stopwords
import re

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import transformers
from transformers import BertForSequenceClassification, AdamW, BertConfig, BertTokenizer, get_linear_schedule_with_warmup

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
train_data = pd.read_csv("/content/Text classification train.csv")
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
sw = stopwords.words('english') # Список слов, которые не помогут нам при классификации
#['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves' ... ]

# Функция для очистки текста
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z?.!,¿]+", " ", text)
    text = re.sub(r"http\S+", "",text)
    text = re.sub(r"http", "",text)
    html=re.compile(r'<.*?>')

    text = html.sub(r'',text)

    punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`" + '_'
    for p in punctuations:
        text = text.replace(p,'')

    text = [word.lower() for word in text.split() if word.lower() not in sw]

    text = " ".join(text)

    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"
                           u"\U0001F300-\U0001F5FF"
                           u"\U0001F680-\U0001F6FF"
                           u"\U0001F1E0-\U0001F1FF"
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)

    return text

In [6]:
# Очищаем данные
train_data['text'] = train_data['text'].apply(lambda x: clean_text(x))

In [7]:
X = train_data.text.values
y = train_data.target.values

Вызовем токенизатор из библиотеки transformers. Он будет обучен уже на основе указанного типа - bert-base-uncased. Тип обозначает, что мы будем использовать стандартный BERT из статьи(например, можно указать вместо него RoBERTa).

In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [9]:
print('Исходная последовательность: ', X[1])
print('Токенизация: ', tokenizer.tokenize(X[1]))
print('ID токенов: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(X[0])))

Исходная последовательность:  forest fire near la ronge sask canada
Токенизация:  ['forest', 'fire', 'near', 'la', 'ron', '##ge', 'sas', '##k', 'canada']
ID токенов:  [15616, 3114, 8372, 2089, 16455, 9641, 2149]


In [10]:
# Рассчитаем максимальную длину предложения
max_len = 0

for sent in X:
    input_ids = tokenizer.encode(sent, add_special_tokens=True)
    max_len = max(max_len, len(input_ids))

print('Максимальная длина предложения: ', max_len)

Максимальная длина предложения:  45


In [11]:
input_ids = []
padding_masks = []

for sequence in X:
    # `encode_plus` состоит из следующих этапов:
    #   (1) Разделяем на слова.
    #   (2) Добавляем `[CLS]` токен в начале.
    #   (3) Добавляем `[SEP]` токен в конце.
    #   (4) Токенизируем.
    #   (5) Обрезаем до max_len и добавляем padding.
    #   (6) Получаем padding_mask для `[PAD]` токенов.
    encoded_dict = tokenizer.encode_plus(
                        sequence,
                        add_special_tokens = True, # добавляем '[CLS]' и '[SEP]'
                        max_length = max_len,      # указываем макс. длину
                        pad_to_max_length = True,
                        return_attention_mask = True,   # создаем маску внимания для pad
                        return_tensors = 'pt',     # возвращаем тензор
                   )

    # Добавляем токенезированную последовательность в датасет
    input_ids.append(encoded_dict['input_ids'])

    # Добавляем максу внимания для tokens
    padding_masks.append(encoded_dict['attention_mask'])

# Преобразуем датасет в формат torch
input_ids = torch.cat(input_ids, dim=0)
padding_masks = torch.cat(padding_masks, dim=0)
y = torch.tensor(y)
# Вывдем результаты
print('Исходная последовательность: ', X[1])
print("")
print('Токены:', input_ids[1])
print("")
print('Маска:', padding_masks[1])
print("")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Исходная последовательность:  forest fire near la ronge sask canada

Токены: tensor([  101,  3224,  2543,  2379,  2474,  6902,  3351, 21871,  2243,  2710,
          102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0])

Маска: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])



In [12]:
# Создаем класс датасета
dataset = TensorDataset(input_ids, padding_masks, y)

# Разделяем данные на обучающую и валидационную выборки

train_size = int(0.8 * len(dataset))
val_size = len(dataset)  - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} обучающих последовательностей'.format(train_size))
print('{:>5,} валидационных последовательностей'.format(val_size))

6,090 обучающих последовательностей
1,523 валидационных последовательностей


In [13]:
batch_size = 32

train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset),
            batch_size = batch_size
        )

validation_dataloader = DataLoader(
            val_dataset,
            sampler = SequentialSampler(val_dataset),
            batch_size = batch_size
        )

## Создание модели

In [14]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",  # Указываем такой же тип как в токенизаторе (классический берт).
    num_labels = 2,  # Количество классов.
    output_attentions = False,  # Можно получать на выходе ещё резульататы внимания, но нам это не надо.
    output_hidden_states = False,  # Можно получать все выходы на слоях, но это тоже не нужно.
)

model = model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

## Обучающий цикл

In [16]:
optimizer = AdamW(model.parameters(),lr = 2e-5, eps = 1e-8 )

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [17]:
# Авторы BERT рекомендуют использовать 2-4 эпохи, иначе есть риск переобучиться.
epochs = 4

total_steps = len(train_dataloader) * epochs

# Lr scheduler изменяет lr на основе batch или epoch.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

### Вспомогательные функции

In [18]:
# Функция для измерения качества
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [19]:
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

### Запускаем обучение

In [21]:
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
training_stats = []

total_t0 = time.time()

for epoch_i in range(0, epochs):

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Обучение...')

    t0 = time.time()
    total_train_loss = 0
    model.train()
    for step, batch in enumerate(train_dataloader):
        b_X = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_y = batch[2].to(device)

        optimizer.zero_grad()
        output = model(b_X,
                       token_type_ids=None,
                       attention_mask=b_input_mask, # BERT требует на вход padding_mask
                       labels=b_y)

        loss = output.loss
        total_train_loss += loss.item()

        loss.backward()

        # Ограничиваем значения градиентов максимальным значением 1
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

    # Считаем средний loss для данной epoch
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Измеряем затраченное время
    training_time = format_time(time.time() - t0)
    print("")
    print("  Средний loss обучающей выборки: {0:.2f}".format(avg_train_loss))
    print("  Время, затраченное на эпоху: {:}".format(training_time))

    print("")
    print("Валдиация модели...")
    t0 = time.time()

    model.eval()

    total_eval_accuracy = 0
    best_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in validation_dataloader:
        b_X = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_y = batch[2].to(device)

        with torch.no_grad():
            output= model(b_X,
                          token_type_ids=None,
                          attention_mask=b_input_mask,
                          labels=b_y)
        loss = output.loss
        total_eval_loss += loss.item()
        logits = output.logits
        logits = logits.detach().cpu().numpy()
        label_ids = b_y.to('cpu').numpy()
        # сохраняем качество
        total_eval_accuracy += flat_accuracy(logits, label_ids)

    # Измеряем среднее качество
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    avg_val_loss = total_eval_loss / len(validation_dataloader)
    # Измеряем время валидации.
    validation_time = format_time(time.time() - t0)
    if avg_val_accuracy > best_eval_accuracy:
        torch.save(model, 'bert_model')
        best_eval_accuracy = avg_val_accuracy

    # Записываем все статистики для эпохи.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )
print("")
print("Обучение завершено!")

print("Всего было затрачено {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 4 ========
Обучение...

  Средний loss обучающей выборки: 0.29
  Время, затраченное на эпоху: 0:00:51

Валдиация модели...
  Accuracy: 0.83

======== Epoch 2 / 4 ========
Обучение...

  Средний loss обучающей выборки: 0.30
  Время, затраченное на эпоху: 0:00:50

Валдиация модели...
  Accuracy: 0.82

======== Epoch 3 / 4 ========
Обучение...

  Средний loss обучающей выборки: 0.24
  Время, затраченное на эпоху: 0:00:50

Валдиация модели...
  Accuracy: 0.83

======== Epoch 4 / 4 ========
Обучение...

  Средний loss обучающей выборки: 0.22
  Время, затраченное на эпоху: 0:00:50

Валдиация модели...
  Accuracy: 0.83

Обучение завершено!
Всего было затрачено 0:03:43 (h:mm:ss)


In [22]:
input_sequnece = "Omg, I just saw a fire in the forest!"
input_sequnece = clean_text(input_sequnece)
input_sequnece

'omg, saw fire forest'

In [23]:
encoded_dict = tokenizer.encode_plus(
                        input_sequnece,
                        add_special_tokens = True,
                        max_length = max_len,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   )
encoded_dict

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


{'input_ids': tensor([[  101, 18168,  2290,  1010,  2387,  2543,  3224,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}

In [24]:
b_X = encoded_dict['input_ids']
padding_mask = encoded_dict['attention_mask']

In [25]:
model.eval()
b_X = b_X.to(device)
padding_mask = padding_mask.to(device)
pred = np.argmax(model(b_X, attention_mask=padding_mask)['logits'].detach().cpu())
pred

tensor(1)